In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from gensim.models import Word2Vec
import spacy
import numpy as np
import pickle
from tqdm import tqdm
from catboost import CatBoostClassifier
import shap

In [2]:
sentiment_train_data = pd.read_csv("sentiment-train.csv")
sentiment_test_data = pd.read_csv("sentiment-test.csv")
train_x, train_y = sentiment_train_data["text"], sentiment_train_data["sentiment"]
test_x, test_y = sentiment_test_data["text"], sentiment_test_data["sentiment"]
train_x, train_y

(0             I LOVE @Health4UandPets u guys r the best!! 
 1        im meeting up with one of my besties tonight! ...
 2        @DaRealSunisaKim Thanks for the Twitter add, S...
 3        Being sick can be really cheap when it hurts t...
 4          @LovesBrooklyn2 he has that effect on everyone 
                                ...                        
 59995    best weekend ever; Caty Costigans house, Marle...
 59996    Oh FFS! I've been here all fucking day. Why de...
 59997    Leaving britney-just found out jon from new ki...
 59998    @Neomic I havta' go pee, but Im scared to walk...
 59999    Nooooooooooooooo!!!!!! School today. But the w...
 Name: text, Length: 60000, dtype: object,
 0        1
 1        1
 2        1
 3        1
 4        1
         ..
 59995    0
 59996    0
 59997    0
 59998    0
 59999    0
 Name: sentiment, Length: 60000, dtype: int64)

In [3]:
firstCv = CountVectorizer(max_features=1000)
firstVectorizedTrain = firstCv.fit_transform(train_x)
firstVectorizedTest = firstCv.transform(test_x)
firstVectorizedTrain

<60000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 529271 stored elements in Compressed Sparse Row format>

In [4]:
mnb = MultinomialNB()
mnb.fit(firstVectorizedTrain, train_y)

MultinomialNB()

In [5]:
preds = mnb.predict(firstVectorizedTest)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       177
           1       0.71      0.79      0.75       182

    accuracy                           0.73       359
   macro avg       0.73      0.73      0.73       359
weighted avg       0.73      0.73      0.73       359



In [6]:
binary_cv = CountVectorizer(max_features = 1000, binary = True)
binaryVectorizedTrain = binary_cv.fit_transform(train_x)
binaryVectorizedTest = binary_cv.transform(test_x)
binary_mnb = MultinomialNB()
binary_mnb.fit(binaryVectorizedTrain, train_y)

MultinomialNB()

In [7]:
binaryPreds = binary_mnb.predict(binaryVectorizedTest)
print(classification_report(test_y, binaryPreds))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71       177
           1       0.71      0.80      0.75       182

    accuracy                           0.74       359
   macro avg       0.74      0.73      0.73       359
weighted avg       0.74      0.74      0.73       359



In [8]:
log_reg = LogisticRegression(max_iter = 10**6)
log_reg.fit(firstVectorizedTrain, train_y)
logPreds = log_reg.predict(firstVectorizedTest)
print(classification_report(test_y, logPreds))

              precision    recall  f1-score   support

           0       0.82      0.68      0.75       177
           1       0.74      0.86      0.79       182

    accuracy                           0.77       359
   macro avg       0.78      0.77      0.77       359
weighted avg       0.78      0.77      0.77       359



In [9]:
bin_log_reg = LogisticRegression(max_iter = 10**6)
bin_log_reg.fit(binaryVectorizedTrain, train_y)
binLogPreds = bin_log_reg.predict(binaryVectorizedTest)
print(classification_report(test_y, binLogPreds))

              precision    recall  f1-score   support

           0       0.81      0.67      0.73       177
           1       0.72      0.85      0.78       182

    accuracy                           0.76       359
   macro avg       0.77      0.76      0.76       359
weighted avg       0.77      0.76      0.76       359



In [10]:
skf = StratifiedKFold(n_splits = 10)
for n_features in [1000, 2000, 3000, 4000]:
    without_binary = 0
    with_binary = 0
    for train_index, test_index in skf.split(train_x, train_y):
        vectorizer = CountVectorizer(max_features = n_features)
        vectorizedTrain = vectorizer.fit_transform(train_x[train_index])
        vectorizedTest = vectorizer.transform(train_x[test_index])
        binVectorizer = CountVectorizer(max_features = n_features, binary = True)
        binVectorizedTrain = binVectorizer.fit_transform(train_x[train_index])
        binVectorizedTest = binVectorizer.transform(train_x[test_index])
        
        model = MultinomialNB()
        model.fit(vectorizedTrain, train_y[train_index])
        preds = model.predict(vectorizedTest)
        without_binary += accuracy_score(train_y[test_index], preds)
        
        binModel = MultinomialNB()
        binModel.fit(binVectorizedTrain, train_y[train_index])
        preds = binModel.predict(binVectorizedTest)
        with_binary += accuracy_score(train_y[test_index], preds)
        
    print("With %d features and not binary: %.3f" % (n_features, without_binary / 10))
    print("With %d features and binary: %.3f" % (n_features, with_binary / 10))

With 1000 features and not binary: 0.738
With 1000 features and binary: 0.740
With 2000 features and not binary: 0.749
With 2000 features and binary: 0.751
With 3000 features and not binary: 0.752
With 3000 features and binary: 0.754
With 4000 features and not binary: 0.753
With 4000 features and binary: 0.755


In [11]:
vectorizer = CountVectorizer(max_features = 4000, binary = True)
vectorizedTrain = vectorizer.fit_transform(train_x)
vectorizedTest = vectorizer.transform(test_x)
model = MultinomialNB()
model.fit(vectorizedTrain, train_y)
preds = model.predict(vectorizedTest)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       177
           1       0.76      0.82      0.79       182

    accuracy                           0.78       359
   macro avg       0.78      0.78      0.78       359
weighted avg       0.78      0.78      0.78       359



In [12]:
nlp = spacy.load("en_core_web_sm")
def tokenize_tweet(x):
    tokens = []
    for sent in nlp(x).sents:
        tokens += [str(token) for token in sent]
    return tokens


In [13]:
def avgVec(model, sentence):
    vec = 0
    n = 0
    for word in sentence:
        try:
            vec += model.wv[word]
            n += 1
        except:
            pass
    if n != 0:
        return vec / n
    else:
        return np.zeros(300)

In [14]:
[x.split(" ") for x in train_x][0]

['I', 'LOVE', '@Health4UandPets', 'u', 'guys', 'r', 'the', 'best!!', '']

In [15]:
'''
tokenized_train = []
for i in tqdm(range(len(train_x))):
    tokenized_train.append(tokenize_tweet(train_x[i]))
    
with open("w2v.txt", "wb") as file:
    pickle.dump(tokenized_train, file)
'''

'\ntokenized_train = []\nfor i in tqdm(range(len(train_x))):\n    tokenized_train.append(tokenize_tweet(train_x[i]))\n    \nwith open("w2v.txt", "wb") as file:\n    pickle.dump(tokenized_train, file)\n'

In [16]:
with open("w2v.txt", "rb") as file:
    tokenized_train = pickle.load(file)

In [17]:
model = Word2Vec(sentences=tokenized_train, vector_size=300)

In [18]:
vectorized_train = [avgVec(model, x) for x in tokenized_train]

In [19]:
log = LogisticRegression(max_iter = 10**6)
log.fit(vectorized_train, train_y)

LogisticRegression(max_iter=1000000)

In [20]:
vectorized_test = [avgVec(model, tokenize_tweet(x)) for x in test_x]

In [21]:
preds = log.predict(vectorized_test)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.70      0.54      0.61       177
           1       0.63      0.77      0.69       182

    accuracy                           0.66       359
   macro avg       0.66      0.66      0.65       359
weighted avg       0.66      0.66      0.65       359



In [22]:
all_train_tweets = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, usecols=[0, 5], encoding='latin-1')
all_train_tweets = all_train_tweets[all_train_tweets[0] != 2]
all_train_tweets.loc[all_train_tweets[0] == 4, 0] = 1 
all_train_tweets


0                                                  5
0        0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1        0  is upset that he can't update his Facebook by ...
2        0  @Kenichan I dived many times for the ball. Man...
3        0    my whole body feels itchy and like its on fire 
4        0  @nationwideclass no, it's not behaving at all....
...     ..                                                ...
1599995  1  Just woke up. Having no school is the best fee...
1599996  1  TheWDB.com - Very cool to hear old Walt interv...
1599997  1  Are you ready for your MoJo Makeover? Ask me f...
1599998  1  Happy 38th Birthday to my boo of alll time!!! ...
1599999  1  happy #charitytuesday @theNSPCC @SparksCharity...

[1600000 rows x 2 columns]

In [23]:
'''
train_x = all_train_tweets[5]
train_y = all_train_tweets[0]
results = ""

skf = StratifiedKFold(n_splits = 10)
for n_features in [1000, 2000, 3000, 4000]:
    without_binary = 0
    with_binary = 0
    for train_index, test_index in skf.split(train_x, train_y):
        vectorizer = CountVectorizer(max_features = n_features)
        vectorizedTrain = vectorizer.fit_transform(train_x[train_index])
        vectorizedTest = vectorizer.transform(train_x[test_index])
        binVectorizer = CountVectorizer(max_features = n_features, binary = True)
        binVectorizedTrain = binVectorizer.fit_transform(train_x[train_index])
        binVectorizedTest = binVectorizer.transform(train_x[test_index])
        
        model = MultinomialNB()
        model.fit(vectorizedTrain, train_y[train_index])
        preds = model.predict(vectorizedTest)
        without_binary += accuracy_score(train_y[test_index], preds)
        
        binModel = MultinomialNB()
        binModel.fit(binVectorizedTrain, train_y[train_index])
        preds = binModel.predict(binVectorizedTest)
        with_binary += accuracy_score(train_y[test_index], preds)
    result1 = "With %d features and not binary: %.3f" % (n_features, without_binary / 10)
    result2 = "With %d features and binary: %.3f" % (n_features, with_binary / 10)
    results += result1 + "\n" + result2 + "\n"
    print(result1)
    print(result2)

with open("bonusResults.txt", "w") as file:
    file.write(results)
'''

'\ntrain_x = all_train_tweets[5]\ntrain_y = all_train_tweets[0]\nresults = ""\n\nskf = StratifiedKFold(n_splits = 10)\nfor n_features in [1000, 2000, 3000, 4000]:\n    without_binary = 0\n    with_binary = 0\n    for train_index, test_index in skf.split(train_x, train_y):\n        vectorizer = CountVectorizer(max_features = n_features)\n        vectorizedTrain = vectorizer.fit_transform(train_x[train_index])\n        vectorizedTest = vectorizer.transform(train_x[test_index])\n        binVectorizer = CountVectorizer(max_features = n_features, binary = True)\n        binVectorizedTrain = binVectorizer.fit_transform(train_x[train_index])\n        binVectorizedTest = binVectorizer.transform(train_x[test_index])\n        \n        model = MultinomialNB()\n        model.fit(vectorizedTrain, train_y[train_index])\n        preds = model.predict(vectorizedTest)\n        without_binary += accuracy_score(train_y[test_index], preds)\n        \n        binModel = MultinomialNB()\n        binModel.f

In [24]:
with open("bonusResults.txt", "r") as file:
    print(file.read())

With 1000 features and not binary: 0.745
With 1000 features and binary: 0.746
With 2000 features and not binary: 0.758
With 2000 features and binary: 0.760
With 3000 features and not binary: 0.763
With 3000 features and binary: 0.764
With 4000 features and not binary: 0.766
With 4000 features and binary: 0.767



In [25]:
#Training on the entire data set with 4000 and binary
train_x = all_train_tweets[5]
train_y = all_train_tweets[0]
vectorizer = CountVectorizer(max_features = 4000, binary = True)
model = MultinomialNB()
train = vectorizer.fit_transform(train_x)
test = vectorizer.transform(test_x)
model.fit(train, train_y)
preds = model.predict(test)
print(classification_report(test_y, preds))
#Slight increase to .79 accuracy


              precision    recall  f1-score   support

           0       0.79      0.79      0.79       177
           1       0.80      0.80      0.80       182

    accuracy                           0.79       359
   macro avg       0.79      0.79      0.79       359
weighted avg       0.79      0.79      0.79       359



In [27]:
features = firstCv.get_feature_names()
features[5:15]

['17', '1st', '20', '22', '24', '25', '2nd', '30', 'able', 'about']

In [28]:
goodIndex = features.index('good')
badIndex = features.index('bad')
goodCoef = log_reg.coef_[0][goodIndex]
badCoef = log_reg.coef_[0][badIndex]

goodIndex, goodCoef, badIndex, badCoef

(351, 0.8412336979611966, 80, -0.9913083860578953)

In [29]:
#Most positive and most negative word
features[np.argmax(log_reg.coef_[0])], max(log_reg.coef_[0]), features[np.argmin(log_reg.coef_[0])], min(log_reg.coef_[0])

('welcome', 1.8494054242603981, 'sad', -2.8707243271601968)

In [30]:
train_x, train_y = sentiment_train_data["text"], sentiment_train_data["sentiment"]
test_x, test_y = sentiment_test_data["text"], sentiment_test_data["sentiment"]
model = CatBoostClassifier()
model.fit(firstVectorizedTrain, train_y, verbose=False)
train_preds = model.predict(firstVectorizedTrain)
print(classification_report(train_y, train_preds))
preds = model.predict(firstVectorizedTest)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.80      0.75      0.78     30000
           1       0.77      0.82      0.79     30000

    accuracy                           0.79     60000
   macro avg       0.79      0.79      0.79     60000
weighted avg       0.79      0.79      0.79     60000

              precision    recall  f1-score   support

           0       0.83      0.67      0.74       177
           1       0.73      0.86      0.79       182

    accuracy                           0.77       359
   macro avg       0.78      0.76      0.76       359
weighted avg       0.78      0.77      0.76       359



In [31]:
explainer = shap.TreeExplainer(model, feature_names=firstCv.get_feature_names())
shap_values = explainer(firstVectorizedTrain)
shap.plots.beeswarm(shap_values)

In [32]:
print(train_x[3])
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(firstVectorizedTrain)
shap.plots.force(explainer.expected_value, shap_values[3], firstVectorizedTrain.toarray()[3], firstCv.get_feature_names())

Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup
